<a href="https://colab.research.google.com/github/SSYT01/Stock-Analysis/blob/main/SMA%2BRSI_Trading_Strategy_%26_Backtesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
def RSI(array, n):
  """Relative Strength index"""
  gain = pd.Series(array).diff()
  loss = gain.copy()
  gain[gain < 0 ] = 0
  loss[loss > 0] = 0
  rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
  return 100 - 100 / (1 + rs)

def SMA(array, n):
    """Simple moving average"""
    return pd.Series(array).rolling(n).mean()


Buy the position when:
weekly RSI(30) ≥ daily RSI(30) > 70
Close > MA(10) > MA(20) > MA(50) > MA(100)


Close the position when:
Daily close is more than 2% below MA(10)
8% fixed stop loss is hit

In [18]:
! pip install backtesting

     |████████████████████████████████| 174 kB 11.1 MB/s 
  Created wheel for backtesting: filename=Backtesting-0.3.2-py3-none-any.whl size=173671 sha256=615a3e566d48cce30a797f64e96ad62ef0ac4623164740540235540f61e32409
  Stored in directory: /root/.cache/pip/wheels/22/0e/30/e458272563bdbea1905391b5bb5702e2001a769544d9cb2df3
Successfully built backtesting


In [19]:
from backtesting import Strategy, Backtest
from backtesting.lib import resample_apply


class System(Strategy):
    d_rsi = 30  # Daily RSI lookback periods
    w_rsi = 30  # Weekly
    level = 70
    
    def init(self):
        # Compute moving averages the strategy demands
        self.ma10 = self.I(SMA, self.data.Close, 10)
        self.ma20 = self.I(SMA, self.data.Close, 20)
        self.ma50 = self.I(SMA, self.data.Close, 50)
        self.ma100 = self.I(SMA, self.data.Close, 100)
        
        # Compute daily RSI(30)
        self.daily_rsi = self.I(RSI, self.data.Close, self.d_rsi)
        
        # To construct weekly RSI, we can use `resample_apply()`
        # helper function from the library
        self.weekly_rsi = resample_apply(
            'W-FRI', RSI, self.data.Close, self.w_rsi)
        
        
    def next(self):
        price = self.data.Close[-1]
        
        # If we don't already have a position, and
        # if all conditions are satisfied, enter long.
        if (not self.position and
            self.daily_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.daily_rsi[-1] and
            self.ma10[-1] > self.ma20[-1] > self.ma50[-1] > self.ma100[-1] and
            price > self.ma10[-1]):
            
            # Buy at market price on next open, but do
            # set 8% fixed stop loss.
            self.buy(sl=.92 * price)
        
        # If the price closes 2% or more below 10-day MA
        # close the position, if any.
        elif price < .98 * self.ma10[-1]:
            self.position.close()

/usr/local/lib/python3.7/dist-packages/backtesting/_plotting.py:47: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [28]:
! pip install yfinance

In [29]:
import yfinance as yf
df = yf.download("AAPL",start="2018-01-01", end="2021-10-31",auto_adjust = True)

from backtesting import Backtest

backtest = Backtest(df, System, commission=.002)
stats= backtest.run()

[*********************100%***********************]  1 of 1 completed


In [30]:
%%time

backtest.optimize(d_rsi=range(10, 35, 5),
                  w_rsi=range(10, 35, 5),
                  level=range(30, 80, 10))

Backtest.optimize:   0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 164 ms, sys: 18.6 ms, total: 183 ms
Wall time: 8.25 s


Start                     2018-01-02 00:00:00
End                       2021-10-29 00:00:00
Duration                   1396 days 00:00:00
Exposure Time [%]                     4.76684
Equity Final [$]                      11868.7
Equity Peak [$]                       13256.6
Return [%]                            18.6867
Buy & Hold Return [%]                 263.167
Return (Ann.) [%]                     4.57535
Volatility (Ann.) [%]                 7.51118
Sharpe Ratio                         0.609138
Sortino Ratio                        0.935678
Calmar Ratio                         0.436998
Max. Drawdown [%]                    -10.4699
Avg. Drawdown [%]                    -2.37196
Max. Drawdown Duration      423 days 00:00:00
Avg. Drawdown Duration       74 days 00:00:00
# Trades                                    2
Win Rate [%]                              100
Best Trade [%]                        11.6193
Worst Trade [%]                       6.42126
Avg. Trade [%]                    

In [31]:
backtest.plot()

Row(id='3552', ...)

In [32]:
#These are the trades that would executed with this strategy.
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,142,498,520,70.373585,77.118541,957.783698,0.095845,2019-12-24,2020-01-28,35 days
1,138,522,524,79.235495,75.070928,-574.710310,-0.052559,2020-01-30,2020-02-03,4 days
2,95,651,674,108.456393,119.302670,1030.396301,0.100006,2020-08-04,2020-09-04,31 days
